In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "calgary_canada.osm"  # Replace this with your osm file
SAMPLE_FILE = "calg.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')


使用迭代解析处理地图文件，找出有什么样的标记及其个数。

In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET
import pprint

def count_tags(filename):
        
        elem_dict={}
        for _, elem in ET.iterparse(filename,events=("start",)):
            if elem.tag in elem_dict:
                elem_dict[elem.tag]+=1
            else:
                elem_dict[elem.tag]=1
        return elem_dict


if __name__ == "__main__":
    tags = count_tags('calgary_canada.osm')
    print (tags)
    

{'node': 904169, 'nd': 1095426, 'bounds': 1, 'member': 41473, 'tag': 518065, 'osm': 1, 'way': 122435, 'relation': 4845}


In [4]:
import xml.etree.ElementTree as ET
import pprint
#OSM_FILE = "calgary_canada.osm"  # Replace this with your osm file
SAMPLE_FILE = "calg.osm"
filename=SAMPLE_FILE     
with open (filename,"r") as f:
    phone= set()
    for event, elem in ET.iterparse(filename, events=("start",)):

        if elem.tag == "way" or elem.tag=="node":
            for tag in elem.iter("tag"):
                if tag.attrib['k']=='phone':
                    phone=tag.attrib['v']
                    print phone
                
                       
    

+1.4032756577
+1 403 299 2600
403 228 5553
403-244-2333
+1-403-276-3313
+1-403-457-6011
+1-403-244-7676
+1-403-280-3505
+1 403 476 1308
+1-403-263-0766
(403) 278-0492
+1-403-253-9200
+1-403-777-5105
+1-403-777-5140
+1-403-777-5110
+1-800-293-3717
+1-403-299-4329
+1-403-241-7053
+1-403-239-8177
+1 403 228 3668
(403) 467-6719
+1-403-276-8518
+1-403-277-2666
+1-403-444-4444
+14039365855
1-855-800-0742
+1-403-234-1840
403-276-6892
+1 403 984 4143
+1 403-663-8777
+1 403 253 3174
403-203-1789
14039100321
403-266-7062
+14039322154
+1-403-201-4300
+1-403-288-3333
+1-403-289-9412
+1-403-454-2738
1-403-460-0606
+1-403-453-0828
+1-403-931-6007
+1-403-454-6101
+1-587-351-9729
+1-403-253-1555
+1-403-252-2363
+1-403-253-7588
+1 (403) 264-7808
+1-403-258-1827
+1-403-870-8780
+1-403-475-2676
(403) 282-3588
+1-403-454-8778
+1-403-532-0085
+1-403-720-6989
+1-403-249-0858
+1-403-719-8145
+1-403-719-6899
+1-403-271-7755
+1-403-217-5337
+1(403) 606-2158
+1-403-247-4181
(403) 282-0555
+1-403-262-9360
587-35

In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re



lower = re.compile(r'^([a-z]|_)*$')   #表示仅包含小写字母且有效的标记
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')   #表示名称中有冒号的其他有效标记
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        # YOUR CODE HERE
        flag=0
        if re.search(lower,element.attrib["k"]):
            keys["lower"]+=1
            flag=1
        
        if re.search(lower_colon,element.attrib["k"]):
            keys["lower_colon"]+=1
            flag=1
            
        if re.search(problemchars,element.attrib["k"]):
            keys["problemchars"]+=1
            flag=1
        
        if flag==0:
            keys["other"]+=1
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys



#def test():
    # You can use another testfile 'map.osm' to look at your solution
    # Note that the assertion below will be incorrect then.
    # Note as well that the test function here is only used in the Test Run;
    # when you submit, your code will be checked against a different dataset.
    #keys = process_map('sample.osm')
    #pprint.pprint(keys)
    #assert keys == {'lower': 35770, 'lower_colon': 2934, 'other': 133, 'problemchars': 1}


if __name__ == "__main__":
    keys = process_map('calgary_canada.osm')
    
    print keys
                   
    

{'problemchars': 1, 'lower': 430490, 'other': 11577, 'lower_colon': 75997}


整理街道名称

In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "calgary_canada.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Northeast","Northwest","Southeast",
            "Southwest","North","West","Highway","Way","W","N","E","S"]


# UPDATE THIS VARIABLE
mapping = { 
            "St.":"Street",
            "SE":"Southeast",
            "SW":"Southwest",
            "S.W.":"Southwest",
            "NE":"Northeast",
            "N.E.":"Northeast",
            "NW":"Northwest",
            "N.W":"Northwest",
            "N.W.":"Northwest",
            "Blvd":"Boulevard"
            
            
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    
    for word in mapping:
        if name.endswith(word):
            return name.replace(word,mapping[word])

    return name



if __name__ == '__main__':
    st_types = audit(OSMFILE)
    pprint.pprint(dict(st_types))
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name

    
            
    
    

{'1': set(['Edmonton Trail NE #1', 'Highway 1']),
 '105': set(['29 Street SE #105']),
 '108': set(['2 Ave NE #108']),
 '12': set(['4 Street SW #12']),
 '200': set(['West Creek Court 200']),
 '201': set(['Fairmount Dr SE #201']),
 '204A': set(['Township Road  204A']),
 '205A': set(['Township Road  205A']),
 '21': set(['21']),
 '210': set(['3rd St SW Suite 210', 'Township Road  210']),
 '210A': set(['Township Road  210A']),
 '210B': set(['Township Road  210B']),
 '224': set(['Township Road 224']),
 '22X': set(['Highway 22X']),
 '230': set(['12 St NE #230']),
 '232': set(['Township Road  232', 'Township Road 232']),
 '234': set(['Township Road  234']),
 '243': set(['Township Road 243']),
 '254A': set(['Twp Rd 254A']),
 '261': set(['Township Road 261']),
 '262A': set(['Township Road 262A']),
 '293': set(['Range Road 293']),
 '5': set(['1a St SW # 5']),
 '50A': set(['Range Road  50A']),
 '51': set(['Range Road  51']),
 '51A': set(['Range Road  51A', 'Range Road 51A']),
 '52': set(['Range Ro

            '5 Street SE',
            '50 Avenue SE',
            '50 Street SE',
            '500 McKenzie Lake Bay SE',
            '51 Avenue SE',
            '51 Street SE',
            '52 Avenue SE',
            '52 Street SE',
            '53 Avenue SE',
            '53 Street SE',
            '54 Avenue SE',
            '54 Street SE',
            '55 Street SE',
            '56 Avenue SE',
            '56 Street SE',
            '57 Street SE',
            '58 Avenue SE',
            '58 Street SE',
            '58th Ave SE',
            '59 Avenue SE',
            '6 Avenue SE',
            '6 Street SE',
            '60 Street SE',
            '600 McKenzie Lake Bay SE',
            '61 Avenue SE',
            '62 Avenue SE',
            '62 Street SE',
            '64 Avenue SE',
            '66 Avenue SE',
            '66 Street SE',
            '68 Street SE',
            '69 Avenue SE',
            '7 Avenue SE',
            '70 Avenue SE',
            '700 McKenzie Lak

Montgomery Boulevard NW => Montgomery Boulevard Northwest
5A Street NW => 5A Street Northwest
35 Avenue NW => 35 Avenue Northwest
112 Avenue NW => 112 Avenue Northwest
22 Street NW => 22 Street Northwest
1508 Varsity Estates Drive NW => 1508 Varsity Estates Drive Northwest
40 Avenue NW => 40 Avenue Northwest
Northland Drive NW => Northland Drive Northwest
Sherwood Boulevard NW => Sherwood Boulevard Northwest
Edgebrook Rise NW => Edgebrook Rise Northwest
Crowfoot Way NW => Crowfoot Way Northwest
Hawkwood Boulevard NW => Hawkwood Boulevard Northwest
Varsity Estates Circle NW => Varsity Estates Circle Northwest
Memorial Drive NW => Memorial Drive Northwest
Dalmarnock Crescent NW => Dalmarnock Crescent Northwest
Silvermead Road NW => Silvermead Road Northwest
Bowridge Crescent NW => Bowridge Crescent Northwest
Crowfoot Rise NW => Crowfoot Rise Northwest
100 Hampshire Court NW => 100 Hampshire Court Northwest
600 Varsity Estates Place NW => 600 Varsity Estates Place Northwest
19 Street NW =

10th Street SW => 10th Street Southwest
Scarboro Avenue SW => Scarboro Avenue Southwest
Woodlark Drive SW => Woodlark Drive Southwest
Kootenay Steet SW => Kootenay Steet Southwest
11 Avenue SW => 11 Avenue Southwest
8 Ave SW => 8 Ave Southwest
23 Avenue SW => 23 Avenue Southwest
10th Ave SW => 10th Ave Southwest
Shawnee Rise SW => Shawnee Rise Southwest
22 Street SW => 22 Street Southwest
14A Street SW => 14A Street Southwest
Woodside Road SW => Woodside Road Southwest
Somervale Close SW => Somervale Close Southwest
29 Avenue SW => 29 Avenue Southwest
41 Street SW => 41 Street Southwest
107 Avenue SW => 107 Avenue Southwest
Canova Road SW => Canova Road Southwest
Sarcee Road SW => Sarcee Road Southwest
1 Street SW => 1 Street Southwest
500 Killarney Glen Court SW => 500 Killarney Glen Court Southwest
Glenmount Drive SW => Glenmount Drive Southwest
50 Avenue SW => 50 Avenue Southwest
Somervale Dr SW => Somervale Dr Southwest
Bow Trail SW => Bow Trail Southwest
Canfield Crescent SW => Ca

整理电话格式

In [7]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE ="calgary_canada.osm"
a= re.compile(r'^[^+](\d.*){1,}')   #匹配不以“+”开头的电话号码

def audit_phone(invalid_phone,phone):
    
    if a.search(phone):
        invalid_phone.add(phone)

    

def is_phone(elem):
    return (elem.attrib['k']=="phone")




def audit(osmfile):
    osm_file = open(osmfile, "r")
    invalid_phone= set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "way" or elem.tag=="node":
            for tag in elem.iter("tag"):
                if is_phone(tag):
                    audit_phone(invalid_phone, tag.attrib['v'])
   
    osm_file.close()
                
                       
    return invalid_phone
     

b= re.compile(r'^[4|(](\d.*){1,}')   #匹配“4或（”开头的电话号码。
               
def update_phone(phone):
    if b.search(phone):
        phone="+1"+phone
        phone=phone.replace(".","")
        return phone
    else:
        return phone
   
    

if __name__ == '__main__':
    ph = audit(OSMFILE)
    print (ph)
    for phone in ph:
            better_phone = update_phone(phone)
            print phone, "=>", better_phone

set(['(403)286-8452', '(403) 270- 3780', '403 547-5020', '(403) 724-0123', '(403) 873-1015', '(403)932-5588', '(403) 781-6614', '4039788915', '(403) 278-0492', '403 288 6033', '888-381-2180', '(403) 290-0036', '(587) 747-8005', '(403) 662-8500', '(403) 282-5011', '(403)932-2455 ', '403-252-1800', '403-945-2437', '403-543-8188', '(403) 264-3366', '587-777-3378', '403-293-3900', '(403) 257-9700', '(403) 452-3565', '403-978-1520', '(403)291-9888', '(403) 287-6749', '(403)948-8804', '4035362480', '403-226-0010', '403-980-4670', '(587) 471-2480', '(403) 203-9299', '403-253-0101', '14032800880', '(403) 275-4444', '403-912-5115', '403-266-7062', '403-261-0931', '(403) 283-0654', '(403)240-3822', '(403) 474-7407', '(403) 249-7799', '403-246-7298', '(403) 235-2485', '403-202-8507', '403-245-5553 ext 15', '(403) 269-5581', '4032910323', '403 287 8008', '403-203-1789', '403-288-5444', '(587) 352-0452', '403 269 6551', '(403) 516-0347', '(587) 438-8838', '403-697-0868', '(403) 253-3367', '403 228 

整理邮编格式

In [8]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE ="calgary_canada.osm"
d=re.compile(r'([A-Z]\d){3}')   #匹配缺失空格的邮编。

def audit_zipcode(invalid_zipcodes,zipcode):
    
    if d.search(zipcode):
        invalid_zipcodes.add(zipcode)

    

def is_zipcode(elem):
    return (elem.attrib['k']=="addr:postcode")




def audit(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes= set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "way" or elem.tag=="node":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes, tag.attrib['v'])
   
    osm_file.close()
                
                       
    return invalid_zipcodes
     

                
def update_zip(zipcode):
    if d.search(zipcode):
        zipcode=zipcode[:3]+" "+zipcode[3:]
        return zipcode
    else:
        return zipcode
    
        
    
    

if __name__ == '__main__':
    invalid_zipcodes=audit(OSMFILE)
    print invalid_zipcodes
    for zipcode in invalid_zipcodes:
            better_zipcode= update_zip(zipcode)
            print zipcode, "=>", better_zipcode

set(['T2J1Y8', 'T2J1Y9', 'T2N3M4', 'T2N3M5', 'T2N3M2', 'T2N3M3', 'T2N3M6', 'T2N3M1', 'T2N0G4', 'T2N0G3', 'T3A0H7', 'T2N3M8', 'T2J1Y7', 'T2G4H6', 'T1X0K7', 'T3P0J9', 'T3S0B2', 'T2N3M7', 'T2V1P3', 'T2C2S4', 'T2C2S3', 'T3N0A6', 'T1Y6H2', 'T2M3W8', 'T2N3J9', 'T2G4K1', 'T2G4K2', 'T2C0H6', 'T2T0L4', 'T2L1H3', 'T2G0Y4', 'T2Z0H2', 'T3S0A2', 'T3G0C3', 'T3G4K4', 'T2P0W3', 'T3A1S4', 'T2E4N1', 'T2C5P1', 'T2N1X7', 'T2G2Z4', 'T2N2G3', 'T2N2G4', 'T2N2G9', 'T2J3M3', 'T2C2R6', 'T2X0A1', 'T3A2W6', 'T3A0Y9', 'T3A0Y8', 'T2W5J1', 'T2W5G1', 'T3J0G7', 'T3S0B9', 'T3A0Y5', 'T3A0Y7', 'T3A0Y6', 'T2N1G7', 'T2C4S9', 'T1X1L8', 'T2C4S6', 'T3A3B1', 'T2N1G8', 'T2X0W8', 'T2C5M4', 'T1Y6R7', 'T3A1R7', 'T3G2S9', 'T2Z0A9', 'T2C3W5', 'T2M4X6', 'T2M1Z7', 'T3S0A1', 'T2L1H4', 'T2J1X1', 'T1Y6J6', 'T3A5R8', 'T2G0B2', 'T2G0B1', 'T4C0Z8', 'T3R0A1', 'T3G4N1', 'T2X0T9', 'T2G4M4', 'T2Z0B1', 'T2C4Z6', 'T2J1V1', 'T2J1V3', 'T2J1V2', 'T2J1V5', 'T4C0S4', 'T3G4V9', 'T2N2E3', 'T2L2C3', 'T2Y0H5', 'T2Y0H4', 'T2N1V5', 'T2N1V3', 'T2Y5E2', 'T2C2

In [9]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-



import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

#import cerberus

import schema

OSM_PATH = "calgary_canada.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']



def clean_tag(tag):
    if tag.attrib['k']=="addr:street":
        tag.attrib['v']=update_name(tag.attrib['v'],mapping)
    elif tag.attrib['k']=="phone":
        tag.attrib['v']=update_phone(tag.attrib['v'])
    elif tag.attrib['k']=="addr:postcode":
        tag.attrib['v']=update_zip(tag.attrib['v'])
    return tag
        
        
def shape_tag(el,tag):
    tag_cleaned = clean_tag(tag)   #调用清洗好数据的函数
    tag_dict={
        'id':el.attrib['id'],
        'key':tag_cleaned.attrib['k'],       #调用清洗好的数据
        'value':tag_cleaned.attrib['v'],
        'type':'regular'
        }
    if LOWER_COLON.match(tag_dict['key']):
        tag_dict['type'],_,tag_dict['key']=tag_dict['key'].partition(':')
    return tag_dict
    
def shape_way_node(el,i,nd):
    return{
        'id':el.attrib['id'],
        'node_id':nd.attrib['ref'],
        'position':i
    }
    

def shape_element(el, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
#     tags = []  # Handle secondary tags the same way for both node and way elements

   
  
    tags=[shape_tag(el,t) for t in el.iter('tag')]    #从中取出“tag”的内容，把它命名为t。调用shape_tag函数。
    if el.tag == 'node':
        node_attribs={f:el.attrib[f] for f in node_attr_fields}
        return {'node': node_attribs, 'node_tags': tags}
    elif el.tag == 'way':
        way_attribs={f:el.attrib[f] for f in way_attr_fields}
        
        way_nodes=[shape_way_node (el,i,nd)
                  for i,nd in enumerate(el.iter('nd'))]
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


#def validate_element(element, validator, schema=SCHEMA):
    #  """Raise ValidationError if element does not match schema"""
    #if validator.validate(element, schema) is not True:
       # field, errors = next(validator.errors.iteritems())
       # message_string = "\nElement of type '{0}' has the following errors:\n{1}"
       # error_string = pprint.pformat(errors)
        
       # raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'wb') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

      #  validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
           # if el:
              #  if validate is True:
                   # validate_element(el, validator)

            if element.tag == 'node':
                nodes_writer.writerow(el['node'])
                node_tags_writer.writerows(el['node_tags'])
            elif element.tag == 'way':
                ways_writer.writerow(el['way'])
                way_nodes_writer.writerows(el['way_nodes'])
                way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)
